# Plot On/Off Logs
Load and render log files of my concise timestamp to on/off format.

Change the path to the timestamps to on/off logfile in the following cell by changing `log_file = "<path/to/your/logfile>"`.  
You can also specify the width of the spikes via `spike_width`, e.g. by setting `spike_width = pd.Timedelta(hours=1)`.

In [ ]:
import datetime
from typing import Union

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from matplotlib import cm

# change the path to your specificationspd.Timedelta(minutes=5)
log_file: str = "devices.log"
spike_width: Union[pd.Timedelta, float] = pd.Timedelta(minutes=5)


def load_logfile(path_to_logfile: str) -> pd.DataFrame:
    """Load a logfile as a pandas dataframe with a column for each signal."""
    timestamps_to_signals = dict()
    with open(path_to_logfile, "r") as file:
        for line in file:
            words = line.split()
            timestamp = datetime.datetime.strptime(" ".join(words[:-1]), "%Y-%m-%d %H:%M")
            signals = {signal:1 for signal in words[-1]}
            timestamps_to_signals[timestamp] = signals
        df = pd.DataFrame.from_dict(timestamps_to_signals, orient="index")
        return df


def display_graph(df: pd.DataFrame, spike_width: pd.Timedelta = 1):
    fig=plt.figure(figsize=(30, 1.5 * len(df.columns)), dpi=100)
    ax = fig.add_subplot()
    plt.title("Timeline of the On/Off Signals")
    row_height = 1
    for i, (title, series) in enumerate(df.iteritems(), 0):
        ax.broken_barh(
            [(x, spike_width) for x in df.index[df[title] == 1]],
            (i * row_height, row_height), color=cm.brg(1.*i/len(df.columns)))
    
    fig.autofmt_xdate()
    plt.xticks(rotation=30, ha='right')
    ax.grid(True)
    ax.xaxis.set_major_formatter(mpl.dates.DateFormatter("%a, %Y-%m-%d"))
    ax.set_xlabel("Time")
    ax.xaxis.set_major_locator(mpl.dates.DayLocator())
    ax.xaxis.set_minor_locator(mpl.dates.HourLocator())
    ytick_offset = row_height * 0.5
    ax.set_yticks([i + ytick_offset for i in range(len(df.columns))])
    ax.set_yticklabels(df.columns.to_list())
    display(fig)
    plt.close()


df = load_logfile(log_file)
display_graph(df, spike_width)